In [ ]:
import $ivy.`org.typelevel::cats-core:2.1.0`

// These are all the imports you need for everything here
import cats.implicits._
import cats.{Eval, Id, Monad}
import cats.data.{Reader, State, Writer}

import scala.concurrent.Future
import scala.concurrent.ExecutionContext.Implicits.global

# Monads

## What is a Monad?
A simple definition: Something that can be flatMapped

Another definition: A mechanism for sequencing computations

Functors are also for sequencing computations, but are limited in that they only allow one *complication* to occur at the beginning of the sequence.

In [ ]:
def parseInt(str: String): Option[Int] =
  scala.util.Try(str.toInt).toOption

val some = parseInt("10").map(_ * 5).map(_ + 3)
val none = parseInt("Not an int").map(_ * 5).map(_ + 3)

Monads allow to specify what happens next, taking into account an intermediate *complication*.

In [ ]:
def divide(a: Int, b: Int): Option[Int] =
  if (b == 0) None else Some(a / b)

def stringDivideBy(str1: String, str2: String): Option[Int] =
  parseInt(str1).flatMap { num1 =>
    parseInt(str2).flatMap { num2 =>
      divide(num1, num2)
    }
  }

val some  = stringDivideBy("10", "5")
val none1 = stringDivideBy("Not an int", "5")
val none2 = stringDivideBy("10", "Not an int")
val none3 = stringDivideBy("10", "0")

The *complication* in this example is `Option`s fail-fast error handling behavior.

Every `Monad` is also a `Functor` (we'll do the proof in a bit). If we have `flatMap` and `map` we can use for comprehensions to clarify the sequencing behavior:

In [ ]:
def stringDivideBy(str1: String, str2: String): Option[Int] =
  for {
    num1 <- parseInt(str1)
    num2 <- parseInt(str2)
    ans  <- divide(num1, num2)
  } yield ans

## Examples of Monads

### `List`
`List`'s `flatMap` can be thought of as iterating over the elements of the list. This is reinforced by the syntax of for comprehensions, which look very much like imperative for-loops:

In [ ]:
for {
  x <- (1 to 3).toList
  y <- (4 to 5).toList
} yield (x, y)

However, there is another mental model we can apply that highlights the monadic behaviour of `List`. If we think of `List`s as sets of intermediate results, `flatMap` becomes a construct that calculates permutations and combinations.

`List`'s *complication* is calculating all possible intermediate results.

### `Future`
`Future`'s `flatMap` allows us to sequence asynchronous computations without worrying about the complexities of thread pools and schedulers.

In [ ]:
def doSomethingLongRunning: Future[Int] = ???
def doSomethingElseLongRunning: Future[Int] = ???
def doSomethingVeryLongRunning: Future[Int] =
  for {
    result1 <- doSomethingLongRunning
    result2 <- doSomethingElseLongRunning
  } yield result1 + result2

If you're familiar with `Future`, you’ll know that the code above is running each operation *in sequence*. This becomes clearer if we expand out the for comprehension to show the nested calls to `flatMap`:

In [ ]:
def doSomethingVeryLongRunning: Future[Int] =
  doSomethingLongRunning.flatMap { result1 =>
    doSomethingElseLongRunning.map { result2 =>
      result1 + result2
    }
  }

Each `Future` in our sequence is created by a function that receives the result from a previous `Future`. In other words, each step in our computation can only start once the previous step is finished.



`Future`s *can* be ran in parallel, of course, but monads are all about sequencing computations. Also, running `Future`s in parallel shows how `Future`s break **referential transparency**.

## Definition of a Monad
A monad requires slightly more than just `flatMap` a monad needs both:
- `pure` of type `A => F[A]`
- `flatMap` of type `(F[A], A => F[B]) => F[B]`

`pure` abstracts over constructors. Every monad can be created from a plain value. Think of `pure` as an abstraction of `Some(value)` for `Option`, `Future.successful(value)` for `Future`, and `List(value)` for `List`.

Here is a simplified version of the `Monad` type class in Cats:

In [ ]:
// Calling it MyMonad to not overwrite cats.Monad
trait MyMonad[F[_]] {
  def pure[A](value: A): F[A]
  def flatMap[A, B](value: F[A])(func: A => F[B]): F[B]
}

## Monad Laws
Like most type classes, `Monad` has laws:

**Left Identity**

`pure(value).flatMap(f) == f(value)`

In [ ]:
val result1 = "10".some.flatMap(parseInt)
val result2 = parseInt("10")

**Right Identity**

`monad.flatMap(pure) == monad`

In [ ]:
val result1 = "10".some
val result2 = "10".some.flatMap(_.some)

**Associativity**

`monad.flatMap(f).flatMap(g) == monad.flatMap(x => f(x).flatMap(g))`

In [ ]:
def tenDividedBy(a: Int) = divide(10, a)

val result1 = "10".some.flatMap(parseInt).flatMap(tenDividedBy)
val result2 = "10".some.flatMap(x => parseInt(x).flatMap(tenDividedBy))

## Excercise
Lets do an exercise from the book and prove that every `Monad` is also a `Functor`:

In [ ]:
// Calling it MyMonad to not overwrite cats.Monad
trait MyMonad[F[_]] {
  def pure[A](value: A): F[A]
  def flatMap[A, B](value: F[A])(func: A => F[B]): F[B]
  def map[A, B](value: F[A])(func: A => B): F[B] =
    ???
}

Side Note:

This interesting phenonemon where we are severly limited in the number of possible implementations of a polymorphic function is called **parametricity**. There is only one way to write this function and have it compile. Go ahead and try to find another.

## Monads in Cats
### Monad Type Class
The monad type class is `cats.Monad`. It extends two other type classes: `FlatMap`, which provides the `flatMap` method, and `Applicative`, which provides, amongst other things, the `pure` function. `Applicative` also extends `Functor`, which gives every `Monad` a `map` method as we just saw in the exercise.

Here are some examples:

In [ ]:
val opt1 = Monad[Option].pure(3)
val opt2 = Monad[Option].flatMap(opt1)(a => Some(a + 2))
val opt3 = Monad[Option].map(opt2)(a => 100 * a)

val list1 = Monad[List].pure(3)
val list2 = Monad[List].flatMap(List(1, 2, 3))(a => List(a, a * 10))
val list3 = Monad[List].map(list2)(_ + 123)

`Monad` also provides many other userful methods, including all the methods from `Applicative` and `Functor`.

### Default Instances
Cats also provides instances for all the monads in the standard library (`Option`, `List`, `Vector`, etc).

Cats also provides a `Monad` for `Future`. Unlike the methods on the `Future` class itself, the `pure` and `flatMap` methods on the monad can’t accept implicit `ExecutionContext` parameters (because the parameters aren’t part of the
definitions in the `Monad` trait). To work around this, Cats requires us to have an `ExecutionContext` in scope when we summon a `Monad` for `Future`:

In [ ]:
// Need to restart kernel without global ExecutionContext import to show this
val fm = Monad[Future]
// The Monad instance uses the captured ExecutionContext for subsequent calls to pure and flatMap

### Monad Syntax
We can use `pure` to construct instances of a monad. We’ll often need to specify the type parameter to disambiguate the particular instance we want.

In [ ]:
val opt  = 1.pure[Option]
val list = 1.pure[List]

It’s difficult to demonstrate the `flatMap` and `map` methods directly on Scala monads like `Option` and `List`, because they define their own explicit versions of those methods. Instead we’ll write a generic function that performs a calculation on parameters that come wrapped in a monad of the user’s choice:

In [ ]:
def sumSquare[F[_]: Monad](a: F[Int], b: F[Int]): F[Int] =
  for {
    x <- a
    y <- b
  } yield x*x + y*y

sumSquare(3.some, 4.some)

sumSquare(List(1, 2, 3), List(4, 5))

### The Identity Monad
The `sumSquare` method is pretty neat. What if we want to use it with plain old values?

In [ ]:
sumSquare(3, 4)

Quite a lovely error message isn't it? It would be awesome if we could use it with monadic and non-monadic code. Fortunately, Cats provides the `Id` type to bridge the gap:

In [ ]:
sumSquare[Id](3, 4)

Interesting. Lets take a look at the definition of `Id`:

In [ ]:
// Calling it MyId to not overwrite cats.Id
type MyId[A] = A

It's just a type alias to turn a type into a single-parameter type constructor. You can cast anything to the corresponding `Id`. Cats provides instances for various type classes for `Id`, including `Functor` and `Monad`:

In [ ]:
val stringId = "Dave" : Id[String]
val intId    = 123 : Id[Int]
val listId   = List(1, 2, 3) : Id[List[Int]]

val a = Monad[Id].pure(3)
val b = Monad[Id].flatMap(a)(_ + 1)

val c =
  for {
    x <- a
    y <- b
  } yield x + y

### Exercise
Lets try to implement `Monad` for `Id`:

In [ ]:
// Ignore StackSafeMonad for now
implicit val monadForId: Monad[Id] = new Monad[Id] with cats.StackSafeMonad[Id] {
  override def pure[A](value: A): Id[A] = value
  override def map[A, B](initial: Id[A])(func: A => B): Id[B] = func(initial)
  override def flatMap[A, B](initial: Id[A])(func: A => Id[B]): Id[B] = func(initial)
}

This ties in with our understanding of functors and monads as sequencing type classes. Each type class allows us to sequence operations ignoring some kind of complication. In the case of `Id` there is no complication, making `map` and
`flatMap` the same thing.

### Either
In 2.11 and earlier, `Either` wasn't really a monad since it didn't have `map` and `flatMap` methods. In 2.12 it became *right biased*. Now `Either` makes the decision that the right side represents the success case and supports `map` and `flatMap` directly. Cats back-ports this behavior to 2.11 by implementing `Monad` for `Either`, allowing you to call `map` and `flatMap` on it as you would in 2.12.

### Either Syntax
You can create instances of `Either` with `asRight` and `asLeft`:

In [ ]:
val a = 3.asRight[String]
val b = 4.asRight[String]

val result = for {
  x <- a
  y <- b
} yield x*x + y*y

These "smart constructors" have advantages over `Left.apply` and `Right.apply`, similar to the advantages of `value.some` over `Some(value)`. They return type `Either` instead of `Left` and `Right` which helps avoid inference bugs cause by over-narrowing.

Cats adds useful extension methods to `Either` and the `Either` companion object:

In [ ]:
val a = Either.catchOnly[NumberFormatException]("foo".toInt)
val b = Either.catchNonFatal(sys.error("Badness"))

// We can also create Either form other types
val fromTry = Either.fromTry(scala.util.Try("foo".toInt))
val fromOpt = Either.fromOption[String, Int](None, "Badness")

// getOrElse and orElse are added
val getOrElse = "Error".asLeft[Int].getOrElse(0)
val orElse    = "Error".asLeft[Int].orElse(2.asRight[String])

// ensure allows us to check a predicate and return Left if it isn't satisfied
val ensure = -1.asRight[String].ensure("Must be non-negative!")(_ > 0)

// We get recover and recoverWith, similar to Future
val recover = "error".asLeft[Int].recover {
  case str => -1
}

val recoverWith = "error".asLeft[Int].recoverWith {
  case str => Right(-1)
}

// We get leftMap and bimap to compliment map
val leftMap = "foo".asLeft[Int].leftMap(_.reverse)
val bimap1  = 6.asRight[String].bimap(_.reverse, _ * 7)
val bimap2  = "bar".asLeft[Int].bimap(_.reverse, _ * 7)

// Swap lets up switch the left and right
val swap = 123.asRight[String].swap

// Finally, there is a bunch of conversion methods
val toOpt  = a.toOption
val toList = a.toList
val toTry  = a.toTry
val toVal  = a.toValidated

### Error Handling and MonadError
The book says this chapter is optional so I'll give it a quick summary. You can abstract over error handling behavior similar to that seen in `Either`, `Try`, and `Future` with the type class `MonadError`. Here is a simplified definition if you're curious:

In [ ]:
// Calling it MyMonadError to not overwrite cats.MonadError
trait MyMonadError[F[_], E] extends Monad[F] {
  def raiseError[A](e: E): F[A]
  def handleErrorWith[A](fa: F[A])(f: E => F[A]): F[A]
  def ensure[A](fa: F[A])(error: => E)(predicate: A => Boolean): F[A] =
    flatMap(fa)(a => if (predicate(a)) pure(a) else raiseError(error))
  // And plenty more utility methods...
}

### The Eval Monad
`cats.Eval` is a monad that allows us to abstract over different *models of evaluation*. It abstracts over 3 models *eager*, *lazy*, and *memoized*.

In [ ]:
// Eager and memoized
val x = {
  println("Computing X")
  math.random
}

In [ ]:
x

In [ ]:
// Lazy and not memoized
def y = {
  println("Computing Y")
  math.random
}

In [ ]:
y

In [ ]:
lazy val z = {
  println("Computing Z")
  math.random
}

In [ ]:
z

`Eval` has three subtypes: `Now`, `Later`, and `Always`.

In [ ]:
val now = Eval.now {
  println("Computing now")
  math.random
}

val later = Eval.later {
  println("Computing later")
  math.random
}

val always = Eval.always {
  println("Computing always")
  math.random
}

In [ ]:
val nowValue = now.value

In [ ]:
val laterValue = later.value

In [ ]:
val alwaysValue = always.value

Like all monads, `Eval`'s `map` and `flatMap` methods add computations to a chain. In this case, however, the chain is stored explicitly as a list of functions. The functions aren’t run until we call `Eval`'s `value` method to request a result:

In [ ]:
val greeting = Eval.
  now { println("Step 1"); "Hello" }.
  map { str => println("Step 2"); s"$str world" }

In [ ]:
greeting.value

Note that, while the semantics of the originating `Eval` instances are maintained, mapping and flatMapping functions are always caled lazily and on demand (`def` semantics).

`Eval` has a `memoize` method that allows us to memoize a chain of computations. The result of the chain up to the call to `memoize` is cached, whereas calculations after the call retain their original semantics:

In [ ]:
val saying = Eval.
  always { println("Step 1"); "The cat" }.
  map { str => println("Step 2"); s"$str sat on" }.
  memoize.
  map { str => println("Step 3"); s"$str the mat" }

In [ ]:
saying.value

A useful property of `Eval` is that it's `map` and `flatMap` methods are *trampolined*. This means that we can nest calls to `map` and `flaMap` without consuming stack frames or blowing the stack. `Eval` is stack safe.

In [ ]:
def factorial(n: BigInt): BigInt =
  if(n == 1) n else n * factorial(n - 1)

factorial(50000)

In [ ]:
// Let's try that again
def factorial(n: BigInt): Eval[BigInt] =
  if(n == 1) {
    Eval.now(n)
  } else {
    factorial(n - 1).map(_ * n)
  }

factorial(50000).value

In [ ]:
def factorial(n: BigInt): Eval[BigInt] =
  if(n == 1) {
    Eval.now(n)
  } else {
    Eval.defer(factorial(n - 1).map(_ * n))
  }

factorial(50000).value

`Eval.defer` takes an existing instance of `Eval` and defers its evaluation. The `defer` method is trampolined like `map` and `flatMap`, so we can use it as a quick way to make an existing operation stack safe. We must remember that trampolining is not free. It avoids consuming stack by creating a chain of function objects on the head. There is still a limit, it's just based on the size of the heap rather than the stack.

### Exercise
Let's make `foldRight` stack safe using `Eval`:

In [ ]:
def foldRight[A, B](as: List[A], acc: B)(fn: (A, B) => B): B =
  as match {
    case head :: tail =>
      fn(head, foldRight(tail, acc)(fn))
    case Nil =>
      acc
  }

val list = List.fill(50000)(1)

foldRight(list, 0)(_ + _)

### The Writer Monad
`cats.data.Writer` is a monad that lets us carry a log along with the computation and extract the log along with the final result. A common use for `Writer` is recording sequences of steps in a multi-threaded computation where standard imperative logging techniques can result in interleaved messages from different contexts.

A `Writer[W, A]` carries two values, the log of type `W` and a result of type `A`.

In [ ]:
// We can create a writer with logs and a result using its constructor
val writer1 = Writer(Vector(
  "It was the best of times",
  "it was the worst of times"
), 1859)

// Or we can use this syntax
val writer2 = 1859.writer(Vector("It was the best of times", "it was the worst of times"))

// Or we can create it using the pure syntax, as long as we have Monoid[W] in scope to create an empty log
type Logged[A] = Writer[Vector[String], A]
val writerNoLog = 123.pure[Logged]

// We can create a writer with a log but no result
val writerNoResult = Vector("msg1", "msg2", "msg3").tell

// We can extract the result and log using .value and .written respectively
val log1 = writer1.written
val result1 = writer1.value

// Or we can get both at once using .run
val (log2, result2) = writer1.run

// We can use map and flatMap since it's a monad
val writer3 = for {
  a <- 10.pure[Logged]
  _ <- Vector("a", "b", "c").tell
  b <- 32.writer(Vector("x", "y", "z"))
} yield a + b

val result3 = writer3.run

// We can use mapWritten to map on the logs
val mapWritten = writer1.mapWritten(_.map(_.toUpperCase)).run

// We can transform both the logs and result at once using bimap or mapBoth
val bimap = writer1.bimap(
  log => log.map(_.toUpperCase),
  res => res * 100
).run

val mapBoth = writer1.mapBoth { (log, res) =>
  val log2 = log.map(_.toUpperCase)
  val res2 = res * 100
  (log2, res2)
}.run

// We can clear the log with reset
val reset = writer1.reset.run

// And we can swap the log and result with swap
val swap = writer1.swap.run

**TODO: Add the Writer exercise?**

### The Reader Monad
`cats.data.Reader` allows us to sequence a bunch of operations that all depend on one input. A common use is dependency injection. If we have a bunch of operations that all depend on some configuration, we can chain them together using a `Reader` to produce one large operation that accepts the configuration as a parameter.

In [ ]:
// We can create a Reader from a function A => B using Reader.apply
case class Cat(name: String, favoriteFood: String)

val catNameReader: Reader[Cat, String] =
  Reader(cat => cat.name)

// We can extract the function using .run and call it as usual
val catName = catNameReader.run(Cat("Garfield", "lasagne"))

Pretty simple. So far `Reader` looks just like a wrapper around a function. What advantage do readers give us over raw functions?

The power of `Reader` comes from it's `map` and `flatMap`.

In [ ]:
// map extends the computation by passing its result through a function
val greetKittyReader: Reader[Cat, String] =
  catNameReader.map(name => s"Hello $name")

val greetKitty = greetKittyReader.run(Cat("Heathcliff", "junk food"))

// flatMap allows us to combine readers that depend on the same input type
val feedKittyReader: Reader[Cat, String] =
  Reader(cat => s"Have a nice bowl of ${cat.favoriteFood}")

val greetAndFeedReader: Reader[Cat, String] =
  for {
    greet <- greetKittyReader
    feed <- feedKittyReader
  } yield s"$greet. $feed."

val greetAndFeedGarfield   = greetAndFeedReader(Cat("Garfield", "lasagne"))
val greetAndFeedHeathcliff = greetAndFeedReader(Cat("Heathcliff", "junk food"))

### Exercise
Lets create a complete example of using `Reader` to build a program that accepts configuration as a parameter. The configuration will consist of two databases: a list of valid users and a list of their passwords:

In [ ]:
case class Db(
  usernames: Map[Int, String],
  passwords: Map[String, String]
)

// Lets create a type alias for the Reader that will make the rest of the code shorter
// type DbReader[A] = ???

// Now lets implement these methods
def findUsername(userId: Int): DbReader[Option[String]] =
  ???

def checkPassword(username: String, password: String): DbReader[Boolean] =
  ???

// Finally, lets use these methods to create a checkLogin
def checkLogin(userId: Int, password: String): DbReader[Boolean] =
  ???

// Now lets test it out
val users = Map(
  1 -> "dade",
  2 -> "kate",
  3 -> "margo"
)

val passwords = Map(
  "dade" -> "zerocool",
  "kate" -> "acidburn",
  "margo" -> "secret"
)

val db = Db(users, passwords)

val login1 = checkLogin(1, "zerocool").run(db)
val login2 = checkLogin(1, "crashoverride").run(db)
val login3 = checkLogin(4, "davinci").run(db)

### The State Monad
`cats.data.State` allows us to pass additional state around as part of a computation. It allows us to model mutable state in a purely functional way, without using mutation.

`State[S, A]` represents a function of type `S => (S, A)` where `S` is the type of the state and `A` is the type of the result.

An instance of `State` is a function that does two things:
- transforms an input state to an output state.
- computes a result.

We can "run" the `State` by providing an inital state. `State` uses `Eval` to maintain stack safety.

In [ ]:
val a = State[Int, String] { state =>
  (state, s"The state is $state")
}

// Get the state and the result:
val (state, result) = a.run(10).value

// Get the state, ignore the result:
val state2 = a.runS(10).value

// Get the result, ignore the state:
val result2 = a.runA(10).value

// The power of state comes from combining instances
val step1 = State[Int, String] { num =>
  val ans = num + 1
  (ans, s"Result of step1: $ans")
}

val step2 = State[Int, String] { num =>
  val ans = num * 2
  (ans, s"Result of step2: $ans")
}

val both = for {
  a <- step1
  b <- step2
} yield (a, b)

val (state3, result3) = both.run(20).value

Cats provides several convenience constructors for creating primitive states.

In [ ]:
// .get extracts the state as the result:
val getDemo = State.get[Int].run(10).value

// .set updates the state and returns unit as the result:
val setDemo = State.set[Int](30).run(10).value

// .pure ignores the state and returns a supplied result:
val pureDemo = State.pure[Int, String]("Result").run(10).value

// .inspect extracts the state via a tranformation function:
val inspectDemo = State.inspect[Int, String](_ + "!").run(10).value

// .modify updates the state using an update function:
val modifyDemo = State.modify[Int](_ + 1).run(10).value

We can assemble these building blocks using a for comprehension.

In [ ]:
import State._

val program: State[Int, (Int, Int, Int)] = for {
  a <- get[Int]
  _ <- set[Int](a + 1)
  b <- get[Int]
  _ <- modify[Int](_ + 1)
  c <- inspect[Int, Int](_ * 1000)
} yield (a, b, c)

val (state, result) = program.run(1).value

### Exercise
**TODO**

## Other Monads in Cats
`IO`

An *effect* monad. Better replacement for `Future`. Makes all code referentially transparent. It's *complication* is side-effecting.

`NonEmptyList`

Same as list, but represents a non-empty set of intermediate results.

`Chain`

Alternative to `List` and `Vector` with different performance characteristics.

`Ior`

Inclusive-or. Similar to Either, but also has the ability to be *both*. Can be thought of as `Result`, `Error`, or `ResultAndWarning`. 

`Kleisli`

Same thing as `ReaderT` which is basically an alias for `A => F[B]`. Very handy and not nearly as intimidating as its name.

`Free`

Turn any sealed hierarchy into a monad. Get the result by converting to a real monad at a later time with an *interpreter*.

`And more...`

Many more from popular libraries such as `Stream` from fs2, `ConnectionIO` from doobie, and `Parser` from atto.